<h1 style="text-align: center; color: #E30613;"><b><i>Entraînement avec MARBERT sur les Données de Sentiments</i></b></h1>

<p style="font-size: 18px;">
Ce notebook présente un workflow complet pour l'entraînement et l'évaluation d'un modèle de classification des sentiments en utilisant <span style="color: #28A745;"><b>MARBERT</b></span>, un modèle de langage pré-entraîné.
</p>

### <span style="color: #28A745;">**Objectifs :**</span>
1. Charger et prétraiter les données textuelles.
2. Encoder les étiquettes des sentiments.
3. Utiliser un tokenizer pour préparer les données pour le modèle.
4. Diviser les données en ensembles d'entraînement, de validation et de test.
5. Entraîner le modèle avec des techniques avancées comme le dropout et l'early stopping.
6. Évaluer les performances du modèle à l'aide de métriques comme la matrice de confusion et le rapport de classification.
7. Visualiser les courbes de pertes et d'accuracy pour analyser les performances.

### <span style="color: #28A745;">**Plan du Notebook :**</span>
1. **Introduction et Bibliothèques nécessaires**  
    Importation des bibliothèques et configuration de l'environnement.
    
2. **Chargement et Prétraitement des Données**  
    Nettoyage des données et préparation des étiquettes.

3. **Tokenization et Préparation des Données**  
    Utilisation du tokenizer MARBERT et création des ensembles d'entraînement, de validation et de test.

4. **Entraînement du Modèle**  
    Entraînement avec des techniques comme l'early stopping et le dropout.

5. **Évaluation et Visualisation**  
    Analyse des performances avec des métriques et des visualisations.

6. **Conclusion**  
    Résumé des résultats et perspectives d'amélioration.

# <span style="color: #E30613;">**MARBERT**</span>

## <span style="color: #28A745;">**Bibiliothèques nécessaires**</span>

In [1]:
import torch
from torch.utils.data import DataLoader, Dataset, random_split
from torch.nn import functional as F
from torch import nn
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig, get_scheduler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import numpy as np
import seaborn as sns
import random

## <span style="color: #28A745;">**Utilisation de GPU**</span>

In [2]:
# ✅ Configuration du device GPU uniquement
assert torch.cuda.is_available(), "CUDA GPU is not available."
device = torch.device("cuda")

AssertionError: CUDA GPU is not available.

## <span style="color: #28A745;">**Chargement des Données**</span>

In [ ]:
# 📁 Chargement des données
df = pd.read_csv("/content/Results/Comments_clean.csv").dropna(subset=["Comments"])

## <span style="color: #28A745;">**Encodage des étiquettes**</span>

In [ ]:
# 🎯 Encodage des étiquettes
label_encoder = LabelEncoder()
df["Sentiments_encoded"] = label_encoder.fit_transform(df["Sentiments"])
num_labels = len(label_encoder.classes_)

## <span style="color: #28A745;">**Tokenizer, Split dataset (Train 70%, Test 20%, Val 10%)**</span>

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv2")

class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self): return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(),
            "attention_mask": encoding["attention_mask"].squeeze(),
            "labels": torch.tensor(self.labels[idx], dtype=torch.long)
        }

# ✂️ Split dataset (Train 70%, Test 20%, Val 10%)
dataset = SentimentDataset(df["Comments"].values, df["Sentiments_encoded"].values, tokenizer)
total_size = len(dataset)
train_size = int(0.7 * total_size)
test_size = int(0.2 * total_size)
val_size = total_size - train_size - test_size
train_set, test_set, val_set = random_split(dataset, [train_size, test_size, val_size], generator=torch.Generator().manual_seed(42))

train_loader = DataLoader(train_set, batch_size=16, shuffle=True)
val_loader = DataLoader(val_set, batch_size=16)
test_loader = DataLoader(test_set, batch_size=16)

## <span style="color: #28A745;">**Chargement du modèle avec dropout custom**</span>

In [ ]:
config = AutoConfig.from_pretrained("aubmindlab/bert-base-arabertv2", num_labels=num_labels, hidden_dropout_prob=0.4, output_attentions=True)
model = AutoModelForSequenceClassification.from_pretrained("aubmindlab/bert-base-arabertv2", config=config)
model.to(device)

"""
# Define model names and paths
model_configs = {
    "AraBERT": {
        "name": "aubmindlab/bert-base-arabertv2",
        "tokenizer": AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv2"),
    },
    "MARBERT": {
        "name": "UBC-NLP/MARBERT",
        "tokenizer": AutoTokenizer.from_pretrained("UBC-NLP/MARBERT"),
    },
    "DziriBERT": {
        "name": "alger-ia/dziribert",
        "tokenizer": AutoTokenizer.from_pretrained("alger-ia/dziribert"),
    },
}
"""

## <span style="color: #28A745;">**Optimiseur et Scheduler**</span>

In [ ]:
# Optimiseur et Scheduler
optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)
num_epochs = 10
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=100, num_training_steps=num_training_steps)

## <span style="color: #28A745;">**Entraînement de Modèle**</span>

In [ ]:
# 🔧 Entraînement avec Early Stopping
best_val_loss = float("inf")
patience, patience_counter = 2, 0
train_losses, val_losses = [], []
train_accs, val_accs = [], []

for epoch in range(num_epochs):
    print(f"\n🟢 Epoch {epoch+1}/{num_epochs}")
    model.train()
    total_loss, correct, total = 0, 0, 0

    for batch in tqdm(train_loader, desc="Training"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        total_loss += loss.item()
        preds = torch.argmax(logits, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_loss = total_loss / len(train_loader)
    train_acc = correct / total
    train_losses.append(train_loss)
    train_accs.append(train_acc)

    # Validation
    model.eval()
    val_loss, correct, total = 0, 0, 0
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validation"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()

            preds = torch.argmax(outputs.logits, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    val_loss = val_loss / len(val_loader)
    val_acc = correct / total
    val_losses.append(val_loss)
    val_accs.append(val_acc)

    print(f"✅ Train Loss: {train_loss:.4f} | Acc: {train_acc:.4f} - Val Loss: {val_loss:.4f} | Acc: {val_acc:.4f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "/content/Results/best_model.pt")
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("⛔ Early stopping triggered.")
            break

## <span style="color: #28A745;">**Évaluation finale**</span>

In [ ]:
# Évaluation finale
model.load_state_dict(torch.load("/content/Results/best_model.pt"))
model.eval()
all_preds, all_labels = [], []

for batch in test_loader:
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    labels = batch["labels"].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    preds = torch.argmax(outputs.logits, dim=1)
    all_preds.extend(preds.cpu().numpy())
    all_labels.extend(labels.cpu().numpy())

print("🔍 Rapport de classification:")
print(classification_report(all_labels, all_preds, target_names=label_encoder.classes_))

## <span style="color: #28A745;">**Courbes des pertes**</span>

In [ ]:
# Courbes des pertes
plt.plot(train_losses, label="Train Loss", color='#28A745')  # Vert
plt.plot(val_losses, label="Validation Loss", color='#E30613')  # Rouge
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training and Validation Loss")

plt.legend()
plt.show()

## <span style="color: #28A745;">**Matrice de confusion**</span>

In [ ]:
# Matrice de confusion
cm = confusion_matrix(all_labels, all_preds)
sns.heatmap(cm, annot=True, fmt='d', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_, cmap="Reds")
plt.xlabel("Prédictions")
plt.ylabel("Prédictions")
plt.title("Matrice de confusion")
plt.show()

<h3 style="text-align: center; color: #E30613;"><b><i>Développé par: OUARAS Khelil Rafik</i></b></h3>